In [1]:
import csv
import itertools
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC

C:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
info = {}
with open('id_label_text_small.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        temp = {}
        temp['itemid'] = row['itemid']
        temp['codes'] = row['codes']
        temp['title'] = row['title']
        temp['text'] = row['text']
        info[row['itemid']] = temp
    csvfile.close()

In [3]:
def text_clean(text):
    #tokenisation
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    # stemming
    for i in range(len(tokens)):
        tokens[i] = stemmer.stem(tokens[i])
    # remove stop words
    test_remove_ = [i for i in tokens if i not in stop]
    return test_remove_

def code_clean(code):
    temp = code.replace("[", "").replace("]","").replace("\'", "").replace(" ", "")
    return temp.split(",")

stop = stopwords.words('english') #+ list(string.punctuation)
stemmer = nltk.PorterStemmer()
for itemid, content in info.items():
    content['text'] = text_clean(content['text'])
    content['codes'] = code_clean(content['codes'])

In [4]:
f = open("industry.txt", "r")
industry = f.read()
f.close()
industry_codes = code_clean(industry)

f = open("region.txt", "r")
region = f.read()
f.close()
region_codes = code_clean(region)

f = open("topics.txt", "r")
topics = f.read()
f.close()
topic_codes = code_clean(topics)

In [5]:
def check_code_exist(lis, codes_lis, temp):
    for code in codes_lis:
        if code in lis:
            temp.append(1)
        else:
            temp.append(0)
            
for idx, content in info.items():
    industry_lis = []
    region_lis = []
    topic_lis = []
    check_code_exist(content['codes'], industry_codes, industry_lis)
    check_code_exist(content['codes'], region_codes, region_lis)
    check_code_exist(content['codes'], topic_codes, topic_lis)
    content['industry'] = industry_lis
    content['region'] = region_lis
    content['topic'] = topic_lis

In [6]:
corpus = []
industry_label = []
region_label = []
topic_label = []
all_label = []
i = 0
for idx, content in info.items():
    str_ = ''
    for word in content['text']:
        str_ += word
        str_ += ' '
    str_ += content['title']
    corpus.append(str_)
    industry_label.append(content['industry'])
    region_label.append(content['region'])
    topic_label.append(content['topic'])
    
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
industry_label = np.array(industry_label)
region_label = np.array(region_label)
topic_label = np.array(topic_label)
all_label = np.array(all_label)

train = X[:7000]
train_label = all_label[:7000]
train_label_in = industry_label[:7000]
train_label_re = region_label[:7000]
train_label_to = topic_label[:7000]

test = X[7000:]
test_label = all_label[7000:]
test_label_in = industry_label[7000:]
test_label_re = region_label[7000:]
test_label_to = topic_label[7000:]

In [7]:
dtc_in = DecisionTreeClassifier(criterion = 'gini', max_depth= 20, min_samples_leaf= 2)
dtc_model_in = dtc_in.fit(train, train_label_in)
dtc_prediction_in = dtc_model_in.predict(test)
true = list(itertools.chain.from_iterable(test_label_in))
pred = list(itertools.chain.from_iterable(dtc_prediction_in))
dtc_in_f1 = f1_score(true, pred, average = 'macro')
print(dtc_in_f1)

0.6200283652351444


In [8]:
dtc_to = DecisionTreeClassifier(criterion= 'gini', max_depth= 20, min_samples_leaf= 1)
dtc_model_to = dtc_to.fit(train, train_label_to)
dtc_prediction_to = dtc_model_to.predict(test)
true = list(itertools.chain.from_iterable(test_label_to))
pred = list(itertools.chain.from_iterable(dtc_prediction_to))
dtc_to_f1 = f1_score(true, pred, average = 'macro')
print(dtc_to_f1)

0.8068770098644188


In [9]:
dtc_re = DecisionTreeClassifier(criterion= 'gini', max_depth= 25, min_samples_leaf= 2)
dtc_model_re = dtc_re.fit(train, train_label_re)
dtc_prediction_re = dtc_model_re.predict(test)
true = list(itertools.chain.from_iterable(test_label_re))
pred = list(itertools.chain.from_iterable(dtc_prediction_re))
dtc_re_f1 = f1_score(true, pred, average = 'macro')
print(dtc_re_f1)

0.8754086079590286
